In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [107]:
tf.__version__

'2.9.1'

# Data load and preprocessing

In [2]:
trining_set_path="SportsDataset//train"
validation_set_path="SportsDataset/valid"


In [9]:
training_set_DataGenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                                    rotation_range = 40, 
                                    width_shift_range = 0.2,
                                    height_shift_range = 0.2,
                                    horizontal_flip = True, 
                                    vertical_flip =True

                            ).flow_from_directory(
                        trining_set_path,
                        target_size=(200,200),
                        batch_size=32,
                        class_mode="categorical"
                        )
validation_set_DataGenerator=tf.keras.preprocessing.image.ImageDataGenerator().flow_from_directory(
                                                                                    validation_set_path,
                                                                                    target_size=(200,200),
                                                                                    batch_size=32,
                                                                                    class_mode="categorical"

                                                                                    )

Found 13572 images belonging to 100 classes.
Found 500 images belonging to 100 classes.


# Transfer learning with keras

In [10]:
base_model=tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, weights="imagenet",input_shape=(200,200,3), pooling='max')

43941136/43941136 [==============================] - 189s 4us/step


In [11]:
base_model.trainable=False

In [15]:
model_efn=tf.keras.models.Sequential()
model_efn.add(base_model)
model_efn.add(tf.keras.layers.Dense(128))
model_efn.add(tf.keras.layers.Dense(100,activation='softmax'))


In [16]:
model_efn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb3 (Functional)  (None, 1536)             10783535  
                                                                 
 dense (Dense)               (None, 128)               196736    
                                                                 
 dense_1 (Dense)             (None, 100)               12900     
                                                                 
Total params: 10,993,171
Trainable params: 209,636
Non-trainable params: 10,783,535
_________________________________________________________________


In [17]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=2)

In [18]:
model_efn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [19]:
history_efn=model_efn.fit(training_set_DataGenerator,validation_data=validation_set_DataGenerator,epochs=10,callbacks=[callback],
                    verbose=1)

Epoch 1/10
425/425 [==============================] - 685s 2s/step - loss: 4.3227 - accuracy: 0.3186 - val_loss: 1.6739 - val_accuracy: 0.6280
Epoch 2/10
425/425 [==============================] - 632s 1s/step - loss: 2.3260 - accuracy: 0.5287 - val_loss: 1.0718 - val_accuracy: 0.7380
Epoch 3/10
425/425 [==============================] - 607s 1s/step - loss: 1.8833 - accuracy: 0.5916 - val_loss: 0.9771 - val_accuracy: 0.7440
Epoch 4/10
425/425 [==============================] - 591s 1s/step - loss: 1.6223 - accuracy: 0.6309 - val_loss: 0.8660 - val_accuracy: 0.7700
Epoch 5/10
425/425 [==============================] - 586s 1s/step - loss: 1.4913 - accuracy: 0.6519 - val_loss: 0.8919 - val_accuracy: 0.7740
Epoch 6/10
425/425 [==============================] - 586s 1s/step - loss: 1.3701 - accuracy: 0.6658 - val_loss: 0.7524 - val_accuracy: 0.7960
Epoch 7/10
425/425 [==============================] - 593s 1s/step - loss: 1.2840 - accuracy: 0.6859 - val_loss: 0.7422 - val_accuracy: 0.7900

In [20]:
model_efn.save("MYeffiNet.h5")

In [30]:
labelList=list(training_set_DataGenerator.class_indices.keys())

In [31]:
labelList

['air hockey',
 'ampute football',
 'archery',
 'arm wrestling',
 'axe throwing',
 'balance beam',
 'barell racing',
 'baseball',
 'basketball',
 'baton twirling',
 'bike polo',
 'billiards',
 'bmx',
 'bobsled',
 'bowling',
 'boxing',
 'bull riding',
 'bungee jumping',
 'canoe slamon',
 'cheerleading',
 'chuckwagon racing',
 'cricket',
 'croquet',
 'curling',
 'disc golf',
 'fencing',
 'field hockey',
 'figure skating men',
 'figure skating pairs',
 'figure skating women',
 'fly fishing',
 'football',
 'formula 1 racing',
 'frisbee',
 'gaga',
 'giant slalom',
 'golf',
 'hammer throw',
 'hang gliding',
 'harness racing',
 'high jump',
 'hockey',
 'horse jumping',
 'horse racing',
 'horseshoe pitching',
 'hurdles',
 'hydroplane racing',
 'ice climbing',
 'ice yachting',
 'jai alai',
 'javelin',
 'jousting',
 'judo',
 'lacrosse',
 'log rolling',
 'luge',
 'motorcycle racing',
 'mushing',
 'nascar racing',
 'olympic wrestling',
 'parallel bar',
 'pole climbing',
 'pole dancing',
 'pole vau

In [35]:
import numpy as np
data_test="SportsDataset//test/archery//1.jpg"
test_image = tf.keras.preprocessing.image.load_img(desktop, target_size = (200,200))
test_image = tf.keras.preprocessing.image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model_efn.predict(test_image)
ind=list(result.flatten()).index(max(list(result.flatten())))
print(labelList[ind])

1/1 [==============================] - 0s 199ms/step
table tennis
